In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix

# read the data
raw_data = pd.read_csv("International students Time management data.csv")

# transform y - variable 
raw_data = raw_data.fillna(raw_data.mode().iloc[0])

y_column = raw_data[['7','10', '11', '12', '14', '17']]
y_column = y_column.replace('Strong Agree','Agree')
y_column = y_column.replace('Strong Disagree','Disagree')
raw_data['avg_response'] = y_column.mode(axis = 1)[0]

raw_data = raw_data.loc[raw_data['avg_response'] != "Neither"]
raw_data['label'] = (raw_data['avg_response'] == 'Agree') | (raw_data['avg_response'] == 'Strong Agree')


In [14]:
raw_data.shape

(96, 23)

In [15]:
# Building blocks for pipeline
impute_and_one_hot = Pipeline([
    ('impute', SimpleImputer(strategy= 'most_frequent')),
    ('encode', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

impute_and_ordinal = Pipeline([
    ('impute', SimpleImputer(strategy= 'most_frequent')),
    ('ordinal', OrdinalEncoder(categories=[['<40%', '40%~49%', '50%~59%', '60%~70%','>70%'],
                                           ['S4', 'S3', 'S2', 'S1', 'S0'], 
                                           ['<40%', '40%~49%', '50%~59%', '60%~70%','>70%'],
                                           ['<18', '18-20', '21-25', '26-30', '31-35', '>36']])),
    ('scalar', StandardScaler())
])

categorical_prepr = ColumnTransformer([
    ("impute_and_one_hot", impute_and_one_hot, ['Course']),
    ("impute_and_ordinal", impute_and_ordinal, ['Academic', 'Attendance', 'English', 'Age'])
])

# gridsearch params
params = {
    "sgdclassifier__loss" : ["hinge", "log", "squared_hinge", "modified_huber"],
    "sgdclassifier__alpha" : [0.01, 0.03, 0.1, 0.3],
    "sgdclassifier__penalty" : ["l2", "l1", "none", "elasticnet"],
    "sgdclassifier__learning_rate": ["constant", "optimal", "invscaling", "adaptive"],
    "sgdclassifier__eta0": [0.01, 0.03, 0.1, 0.3]
}


In [16]:
print(X_train)

     Number    Age Gender   Nationality Program  \
45       46  21-25      M         China      PM   
111     112  31-35      F         Korea      PM   
15       16  18-20      M         China      FC   
21       22  18-20      M        Jordan      FC   
52       53  21-25      M         China      PM   
..      ...    ...    ...           ...     ...   
118     119  21-25      F         China      PM   
117     118  21-25      M         China      PM   
64       65  26-30      F         China      PM   
37       38  21-25      M         China      PM   
71       72    <18      F  Saudi Arabia      FC   

                             Course  English Academic Attendance         6  \
45                         Business  60%~70%  50%~59%         S0  Disagree   
111  Social Sciences and Humanities      NaN      NaN        NaN     Agree   
15                         Business     >70%     >70%         S1     Agree   
21         Media and Communications  60%~70%  60%~70%         S0  Disagree 

In [17]:
train_data = pd.concat([X_train.reset_index(), pd.Series(pred_train, name="pred")], axis=1)

train_data['TP'] = (train_data['label'] == True) & (train_data['pred'] == True)
train_data['TN'] = (train_data['label'] == False) & (train_data['pred'] == False)
train_data['FN'] = (train_data['label'] == True) & (train_data['pred'] == False)
train_data['FP'] = (train_data['label'] == False) & (train_data['pred'] == True)

NameError: name 'pred_train' is not defined

In [17]:
actual = train_data['label']
predicted = train_data['pred']

conf_mat = confusion_matrix(actual, predicted) 

print(conf_mat)

[[ 8 23]
 [ 3 38]]


In [18]:
test_data = pd.concat([X_test.reset_index(), pd.Series(pred_test, name="pred")], axis=1)

test_data['TP'] = (test_data['label'] == True) & (test_data['pred'] == True)
test_data['TN'] = (test_data['label'] == False) & (test_data['pred'] == False)
test_data['FN'] = (test_data['label'] == True) & (test_data['pred'] == False)
test_data['FP'] = (test_data['label'] == False) & (test_data['pred'] == True)


In [34]:
actual = test_data['label']
predicted = test_data['pred']

conf_mat = confusion_matrix(actual, predicted) 

print(conf_mat)

[[ 2  4]
 [ 5 13]]


In [36]:
# False Negative: FN/FN+TP
## The probability that a true positive will be missed by the test
## FN is the number of false negatives 
## TP is the number of true positives (FN+TP being the total number of positives).

# False Positive: FP/FP+TN
## FP is the number of false positives
## TN is the number of true negatives (FP+TN being the total number of negatives)
## It's the probability that a positive result will be given when the true value is negative

In [37]:
def FalseNeg(data):
    FNR = data["FN"].sum() / (data["FN"].sum() + data["TP"].sum())
    return FNR

def FalsePos(data):
    FPR = data["FP"].sum() / (data["FP"].sum() + data["TN"].sum())
    return FPR

In [38]:
# SELECT ROWS IN TRAIN DATASET
train_data_china = train_data.loc[train_data["Nationality"] == "China"]
train_data_others = train_data.loc[train_data["Nationality"] != "China"]

# CALCULATE FNR FOR TRAIN DATASET
FNR_china_train = FalseNeg(train_data_china)
FNR_others_train = FalseNeg(train_data_others)

# CALCULATE FPR FOR TRAIN DATASET
FPR_china_train = FalsePos(train_data_china)
FPR_others_train = FalsePos(train_data_others)

# SELECT ROWS IN TEST DATASET
test_data_china = test_data.loc[test_data["Nationality"] == "China"]
test_data_others = test_data.loc[test_data["Nationality"] != "China"]

# CALCULATE FNR FOR TEST DATASET
FNR_china_test =  FalseNeg(test_data_china)
FNR_others_test =  FalseNeg(test_data_others)

# CALCULATE FPR FOR TEST DATASER
FPR_china_test = FalsePos(test_data_china)
FPR_others_test = FalsePos(test_data_others)

print("FNR China train: ", FNR_china_train)
print("FNR Others train: ", FNR_others_train)
print("Student from China have a 5% higher chance to be falsely predicted to have bad time management")
print("\n")
print("FNR China test: ", FNR_china_test)
print("FNR Others test: ",FNR_others_test)
print("Student from other countries have 37.5% higher probability to be wrongly predicted to have bad time management.")
print("\n")
print("FPR China train: ", FPR_china_train)
print("FPR Others train: ", FPR_others_train)
print("Student from China have a 10% higher probability to be predicted to be good at time management when they are not.")
print("\n")
print("FPR China test: ", FPR_china_test)
print("FPR Others test: ",FPR_others_test)
print("Student from other countries are 29.5% more likely to be predicted to be good at time management when they are not.")

FNR China train:  0.06896551724137931
FNR Others train:  0.25
Student from China have a 5% higher chance to be falsely predicted to have bad time management


FNR China test:  0.2
FNR Others test:  0.375
Student from other countries have 37.5% higher probability to be wrongly predicted to have bad time management.


FPR China train:  0.7777777777777778
FPR Others train:  0.46153846153846156
Student from China have a 10% higher probability to be predicted to be good at time management when they are not.


FPR China test:  1.0
FPR Others test:  0.3333333333333333
Student from other countries are 29.5% more likely to be predicted to be good at time management when they are not.


In [39]:
# SELECT ROWS IN TRAIN DATASET
train_data_m = train_data.loc[train_data["Gender"] == "M"]
train_data_f = train_data.loc[train_data["Gender"] == "F"]

# CALCULATE FNR FOR TRAIN DATASET
FNR_male_train = FalseNeg(train_data_m)
FNR_female_train = FalseNeg(train_data_f)

# CALCULATE FPR FOR TRAIN DATASET
FPR_male_train = FalsePos(train_data_m)
FPR_female_train = FalsePos(train_data_f)

# SELECT ROWS IN TEST DATASET
test_data_m = test_data.loc[test_data["Gender"] == "M"]
test_data_f = test_data.loc[test_data["Gender"] == "F"]

# CACULATE FNR FOR TEST DATASET
FNR_male_test = FalseNeg(test_data_m)
FNR_female_test = FalseNeg(test_data_f)

# CACULATE FPR FOR TEST DATASET
FPR_male_test = FalsePos(test_data_m)
FPR_female_test = FalsePos(test_data_f)


print("FNR Males train: ", FNR_male_train)
print("FNR Females train: ", FNR_female_train)
print("Male students have 20% higher probability than female student to be falsely predicted with bad time management.")
print("\n")
print("FNR Males test: ", FNR_male_test)
print("FNR Females test: ", FNR_female_test)
print("There is no difference between male and female students.")
print("\n")
print("FPR Males train: ", FPR_male_train)
print("FPR Females train: ", FPR_female_train)
print("Male students have 38% higher probability than female student to be falsely predicted with good time management.")
print("\n")
print("FPR Males test: ", FPR_male_test)
print("FPR Females test: ", FPR_female_test)
print("Female students have 4% higher probability than male student to be falsely predicted with good time management.")

FNR Males train:  0.25
FNR Females train:  0.0
Male students have 20% higher probability than female student to be falsely predicted with bad time management.


FNR Males test:  0.2222222222222222
FNR Females test:  0.3333333333333333
There is no difference between male and female students.


FPR Males train:  0.7692307692307693
FPR Females train:  0.5555555555555556
Male students have 38% higher probability than female student to be falsely predicted with good time management.


FPR Males test:  1.0
FPR Females test:  0.6
Female students have 4% higher probability than male student to be falsely predicted with good time management.


In [40]:
# SELECT ROWS IN TRAIN DATASET
train_data_20s = train_data.loc[(train_data["Age"] == "18-20") | (train_data["Age"] == "21-25") | (train_data["Age"] == "26-30")]
train_data_other = train_data.loc[(train_data["Age"] == "<18") | (train_data["Age"] == "31-25") | (train_data["Age"] == ">36")]

# CALCULATE FNR FOR TRAIN DATASET
FNR_train_other = train_data_other["FN"].sum() / (train_data_other["FN"].sum() + train_data_other["TP"].sum())
FNR_train_20s = train_data_20s["FN"].sum() / (train_data_20s["FN"].sum() + train_data_20s["TP"].sum())

# CALCULATE FPR FOR TRAIN DATASET
FPR_train_other = train_data_other["FP"].sum() / (train_data_other["FP"].sum() + train_data_other["TN"].sum())
FPR_train_20s = train_data_20s["FP"].sum() / (train_data_20s["FP"].sum() + train_data_20s["TN"].sum())


# SELECT ROWS IN TEST DATASET
test_data_other = test_data.loc[(test_data["Age"] == "<18") | (test_data["Age"] == "31-25") | (test_data["Age"] == ">36")]
test_data_20s = test_data.loc[(test_data["Age"] == "18-20") | (test_data["Age"] == "21-25") | (test_data["Age"] == "26-30")]

# CALCULATE FNR FOR TEST DATASET
FNR_test_other = test_data_other["FN"].sum() / (test_data_other["FN"].sum() + test_data_other["TP"].sum())
FNR_test_20s = test_data_20s["FN"].sum() / (test_data_20s["FN"].sum() + test_data_20s["TP"].sum())

# CALCULATE FPR FOR TEST DATASET
FPR_test_other = test_data_other["FP"].sum() / (test_data_other["FP"].sum() + test_data_other["TN"].sum())
FPR_test_20s = test_data_20s["FP"].sum() / (test_data_20s["FP"].sum() + test_data_20s["TN"].sum())


print("FNR Old/Young train: ", FNR_train_other)
print("FNR 20s train: ", FNR_train_20s)
print("Age groups above/below 20-30, are 10% less likely to be falsely predicted to have bad time management.")
print("\n")
print("FNR Old/Young test: ", FNR_test_other)
print("FNR 20s test: ", FNR_test_20s)
print("Age groups above/below 20-30, are 50% less likely to be falsely predicted to have bad time management.")
print("\n")
print("FPR Old/Young train: ", FPR_train_other)
print("FPR 20s train: ", FPR_train_20s)
print("Age groups between 20-30, are 1% less likely to be falsely predicted to have good time management.")
print("\n")
print("FPR Old/Young test: ", FPR_test_other)
print("FPR 20s test: ", FPR_test_20s)
print("Age groups between 20-30, are 22% more likely to be falsely predicted to have good time management.")

# runtime error because null division

FNR Old/Young train:  0.0
FNR 20s train:  0.1111111111111111
Age groups above/below 20-30, are 10% less likely to be falsely predicted to have bad time management.


FNR Old/Young test:  nan
FNR 20s test:  0.29411764705882354
Age groups above/below 20-30, are 50% less likely to be falsely predicted to have bad time management.


FPR Old/Young train:  0.75
FPR 20s train:  0.6538461538461539
Age groups between 20-30, are 1% less likely to be falsely predicted to have good time management.


FPR Old/Young test:  nan
FPR 20s test:  0.6666666666666666
Age groups between 20-30, are 22% more likely to be falsely predicted to have good time management.


<ipython-input-40-9c422b0388f8>:19: RuntimeWarning: invalid value encountered in longlong_scalars
  FNR_test_other = test_data_other["FN"].sum() / (test_data_other["FN"].sum() + test_data_other["TP"].sum())
<ipython-input-40-9c422b0388f8>:23: RuntimeWarning: invalid value encountered in longlong_scalars
  FPR_test_other = test_data_other["FP"].sum() / (test_data_other["FP"].sum() + test_data_other["TN"].sum())
